In [1]:
import galaxychop as gchop

In [54]:
gal = gchop.read_hdf5("tests/datasets/gal394242.h5")
gal = gal.disassemble()
gal["potential_s"] = None
gal["potential_dm"] = None
gal["potential_g"] = None
gal = gchop.mkgalaxy(**gal)
gal
gal.to_hdf5("big.h5")

In [3]:
import numba as nb
import numpy as np
from galaxychop import constants as const

# =============================================================================
# NUMBA
# =============================================================================

_numba_eager_signature = nb.float32[:](
    nb.float32[:],
    nb.float32[:],
    nb.float32[:],
    nb.float32[:],
    nb.float32,
)


@nb.jit(_numba_eager_signature, nopython=True, parallel=True, fastmath=True)
def _numba_potential(x, y, z, m, softening):
    """ """
    n = len(x)
    potential_energy = np.zeros(n, dtype=nb.float32)
    soft2 = softening * softening

    for i in nb.prange(n):
        pe_i = 0.0
        x_i = x[i]
        y_i = y[i]
        z_i = z[i]

        for j in range(n):
            if i != j:
                dx = x_i - x[j]
                dy = y_i - y[j]
                dz = z_i - z[j]

                dist_sq = dx * dx + dy * dy + dz * dz + soft2
                dist = np.sqrt(dist_sq)

                pe_i = pe_i + m[j] / dist

        potential_energy[i] = pe_i

    return potential_energy


def numba_potential(x, y, z, m, softening):
    """Wrap the Numba implementation of the gravitational potential.

    Parameters
    ----------
    x, y, z : np.ndarray
        Positions of particles. Shape: (n,1).
    m : np.ndarray
        Masses of particles. Shape: (n,1).
    softening : float, optional
        Softening parameter. Shape: (1,).

    Returns
    -------
    np.ndarray : float
        Specific potential energy of particles.

    """
    soft = np.float32(softening)
    epot = _numba_potential(x, y, z, m, soft)

    return epot * const.G, np.asarray

gchop.preproc.potential_energy.POTENTIAL_BACKENDS["numba"] = numba_potential

In [6]:
%%time
galf = gchop.preproc.potential_energy.potential(gal)

CPU times: user 24min 50s, sys: 1.11 s, total: 24min 51s
Wall time: 1min 36s


In [5]:
%%time
galnb = gchop.preproc.potential_energy.potential(gal, backend="numba")

CPU times: user 15min 46s, sys: 651 ms, total: 15min 46s
Wall time: 1min 1s


In [7]:
galf.potential_energy_

(<Quantity [-195789.8  , -196267.   , -195240.12 , ...,  -64160.082,
             -19707.434, -111534.59 ] km2 / s2>,
 <Quantity [-194351.97 , -193951.72 , -193640.14 , ...,  -17671.715,
             -38886.906,  -22311.053] km2 / s2>,
 <Quantity [-189858.36 , -189337.75 , -189174.36 , ...,  -17366.764,
             -23437.795,  -37337.195] km2 / s2>)

In [8]:
galnb.potential_energy_

(<Quantity [-195786.89 , -196264.45 , -195239.16 , ...,  -64160.45 ,
             -19707.195, -111532.234] km2 / s2>,
 <Quantity [-194350.61 , -193950.67 , -193640.3  , ...,  -17671.906,
             -38887.105,  -22311.201] km2 / s2>,
 <Quantity [-189855.62 , -189337.56 , -189171.69 , ...,  -17366.477,
             -23438.283,  -37337.824] km2 / s2>)

In [12]:
np.max(np.concatenate(galf.potential_energy_) - np.concatenate(galnb.potential_energy_))

<Quantity 4.9375 km2 / s2>

In [13]:
gal

<Galaxy stars=37393, dark_matter=155101, gas=80153, potential=False>

## Galaxia chiquita (rubenstein)

In [27]:
gal = gchop.read_hdf5("small.h5")
gal

<Galaxy stars=100, dark_matter=370, gas=390, potential=False>

In [35]:
%%timeit -r 100
gchop.preproc.potential_energy.potential(gal)

3.57 ms ± 284 μs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [36]:
%%timeit -r 100
gchop.preproc.potential_energy.potential(gal, backend="numba")

3.53 ms ± 663 μs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [37]:
%%timeit -r 100
gchop.preproc.potential_energy.potential(gal, backend="numpy")

4.45 ms ± 197 μs per loop (mean ± std. dev. of 100 runs, 100 loops each)


### Probemos errores

In [38]:
results = {}
for backend in ["fortran", "numpy", "numba"]:
    results[backend] = gchop.preproc.potential_energy.potential(gal, backend=backend)

In [53]:
import itertools as it

for b0, b1 in map(sorted, it.combinations(results, 2)):
    res0, res1 = results[b0].potential_energy_, results[b1].potential_energy_
    res0, res1 = np.concatenate(res0), np.concatenate(res1)
    diff = np.abs(res0 - res1)
    print(f">>> {b0} Vs. {b1} <<<")
    print("Mean:", np.mean(diff))
    print("Max:", np.max(diff))
    print("Min:", np.min(diff))
    print()
    

>>> fortran Vs. numpy <<<
Mean: 1.1684849754445281e-09 km2 / s2
Max: 5.122274160385132e-09 km2 / s2
Min: 0.0 km2 / s2

>>> fortran Vs. numba <<<
Mean: 1.1717337100591863e-09 km2 / s2
Max: 5.122274160385132e-09 km2 / s2
Min: 0.0 km2 / s2

>>> numba Vs. numpy <<<
Mean: 1.0017132262563067e-10 km2 / s2
Max: 4.656612873077393e-10 km2 / s2
Min: 0.0 km2 / s2

